# Importing libraries

In [1]:
import pandas as pd
import numpy as np
import os
import psycopg2

# Importing the CSV file

In [2]:
path = os.path.join('dataset', 'youtube_df.csv')
df = pd.read_csv(path)

In [3]:
df.head()

,video_id,channelTitle,title,description,publishedAt,viewCount,likeCount,favoriteCount,commentCount,duration,definition,caption
0,AiGUvp-2W1k,Dr. Glaucomflecken,Turnover Time,Surgery’s Worst Nightmare,2022-05-12,294007,16227,0,485,95.0,hd,False
1,Az-z9xDqihA,Dr. Glaucomflecken,Divide and Conquer,We’re stronger together,2022-05-10,220784,14349,0,639,131.0,hd,False
2,pw_q1msdLmw,Dr. Glaucomflecken,Don’t Mess With Bill’s Mom,NaN,2022-05-08,1155560,136334,0,1025,59.0,hd,False
3,Dz4buD7LP9A,Dr. Glaucomflecken,The Ophthalmologist gets COVID,"Fortunately, Jonathan is there to help",2022-05-08,235650,12188,0,419,100.0,hd,False
4,JaQGOaqc4Rw,Dr. Glaucomflecken,Couples Therapy with Anesthesia and Surgery,It’s time to take down the drape,2022-05-01,330272,19025,0,605,116.0,hd,False


# Cleanning the headers name

In [4]:
df.columns = ["video_id", "channel_title", "title", "description", "published_at",
              "view_count", "like_count","favorite_count", "comment_count", "duration",
              "definition", "caption"]

In [5]:
df.columns

Index(['video_id', 'channel_title', 'title', 'description', 'published_at',
       'view_count', 'like_count', 'favorite_count', 'comment_count',
       'duration', 'definition', 'caption'],
      dtype='object')

# Replacing the data types to match the sql data types

create table youtube_data
(
    video_id            varchar,
    channel_title       varchar,
    title               varchar,
    descripcion         varchar,
    published_at        date,
    view_count          integer,
    like_count          integer,
    favorite_count      integer,
    comment_count       integer,
    duration            float,
    definition          varchar,
    caption             boolean,
    
);

In [7]:
df.dtypes

video_id           object
channel_title      object
title              object
description        object
published_at       object
view_count          int64
like_count          int64
favorite_count      int64
comment_count       int64
duration          float64
definition         object
caption              bool
dtype: object

In [8]:
replacements = {
    'object' : 'varchar',
    'int64'  : 'int',
    'float64': 'float',
    'bool'   : 'boolean',
    'datetime64': 'timestamp',
    'timedelta64[ns]': 'varchar'
}

replacements

{'object': 'varchar',
 'int64': 'int',
 'float64': 'float',
 'bool': 'boolean',
 'datetime64': 'timestamp',
 'timedelta64[ns]': 'varchar'}

In [9]:
col_str = ", ".join("{} {}".format(n, d) for (n, d) in zip(df.columns, df.dtypes.replace(replacements)))
col_str

'video_id varchar, channel_title varchar, title varchar, description varchar, published_at varchar, view_count int, like_count int, favorite_count int, comment_count int, duration float, definition varchar, caption boolean'

# Open a database connection

In [18]:
conn_str = "host=ytdb.cmr9yexuysqw.us-east-1.rds.amazonaws.com \
            dbname='postgres' \
            user='Davu' password='abcde12345'"
conn = psycopg2.connect(conn_str)
cursor = conn.cursor()
print('Opened database successfully')

Opened database successfully


In [19]:
#drop table with the same name
cursor.execute("drop table if exists youtube_data")

# Creating the table

In [20]:
#create table
cursor.execute("create table youtube_data \
(video_id varchar,\
channel_title varchar,\
title varchar,\
description varchar,\
published_at varchar,\
view_count int,\
like_count int,\
favorite_count int,\
comment_count int,\
duration float,\
definition varchar,\
caption boolean)")

In [21]:
#insert values to table

#save df to csv
df.to_csv('dataset/youtube_data.csv', header = df.columns, index=False, encoding='utf-8')

#open the csv file and save it as an object 
my_file = open('dataset/youtube_data.csv', encoding = 'utf-8')
print('file opened in memory')

file opened in memory


# Uploading the data

In [22]:
#upload to db

SQL_STATEMENT = """
COPY youtube_data FROM STDIN WITH
    CSV
    HEADER
    DELIMITER AS ','

"""
cursor.copy_expert(sql=SQL_STATEMENT, file = my_file)
print('file copied to db')

file copied to db


In [23]:
cursor.execute('grant select on table youtube_data to public')
conn.commit()

cursor.close()
print("table youtube_data imported to db completed")

table youtube_data imported to db completed
